In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

NOTE
'Spacy.xlsx' resulted from an older version of the data cleaning process. This code was used to generate association rules on old iterations of the dataset in order to aid in word removal. This process was not done for the final version of the code.

In [2]:
df = pd.read_excel('Spacy.xlsx')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18923 entries, 0 to 18922
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    18922 non-null  object
 1   type    18923 non-null  object
dtypes: object(2)
memory usage: 295.8+ KB


In [4]:
df.head()

,text,type
0,database project affect environment affect pro...,database
1,sql stick production seem like sql stuck produ...,database
2,database medium folder migration stage able re...,database
3,sql server go away error log addition previous...,database
4,location fail graph overall health monitor sin...,database


In [5]:
df= df.dropna()

In [6]:
data = list(df['text'].apply(lambda x:x.split(' ')))

In [7]:
# Transaction Encoding
a = TransactionEncoder()
a_data = a.fit(data).transform(data)
encoded = pd.DataFrame(a_data, columns=a.columns_)

In [8]:
encoded.head()

,aa,aam,abandon,abandonment,abase,abb,abbas,abbreviate,aberrant,abide,...,zipping,zippy,zo,zoa,zoll,zombie,zone,zoo,zookeeper,zoom
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [9]:
encoded2 = encoded.astype(int)

In [10]:
supports = pd.DataFrame({'Word': encoded2.columns,
                       'Num of Docs': encoded2.sum()})

In [11]:
encoded2 = encoded2.astype(bool)

In [12]:
# Apriori Algorithm
min_support = 0.49
frequent_itemsets = apriori(encoded2, min_support=min_support, max_len=3, use_colnames=True)

In [13]:
# Association Rules
min_confidence = 0.01
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=min_confidence)

In [14]:
print(rules.head(10))

     antecedents    consequents  antecedent support  consequent support  \
0      (already)       (affect)            0.577476            0.535831   
1       (affect)      (already)            0.535831            0.577476   
2       (change)       (affect)            0.660395            0.535831   
3       (affect)       (change)            0.535831            0.660395   
4  (engineering)       (affect)            0.963217            0.535831   
5       (affect)  (engineering)            0.535831            0.963217   
6  (environment)       (affect)            0.769158            0.535831   
7       (affect)  (environment)            0.535831            0.769158   
8         (make)       (affect)            0.649984            0.535831   
9       (affect)         (make)            0.535831            0.649984   

    support  confidence      lift  leverage  conviction  zhangs_metric  
0  0.502272    0.869772  1.623220  0.192843    3.564281       0.908683  
1  0.502272    0.937371  1.6

In [15]:
rules.to_csv('assoc_rules.csv', index=False)

In [16]:
supports.to_csv('document_freq.csv', index=False)